In [1]:
import numpy

In [9]:
import pandas as pd

In [7]:
import mne
import os

In [ ]:
# List of participant IDs
participant_ids = [f"s{i:02d}" for i in range(1, 6)]  # Assuming 5 participants

# Dictionary to store EEG data for each participant
eeg_data = {}

# Load EEG data for each participant
for participant_id in participant_ids:
    file_path = f"eegs/{participant_id}.bdf"
    raw = mne.io.read_raw_bdf(file_path, preload=True)
    eeg_data[participant_id] = raw

# Print information about loaded data
for participant_id, raw in eeg_data.items():
    print(f"Participant ID: {participant_id}")
    # print(raw.info)

In [12]:
# List to store trial data for each participant
participant_trials = {participant_id: [] for participant_id in participant_ids}

# Load trial information from the CSV file
csv_file = 'participant_ratings.csv'  # Update with the actual file name
trial_info = pd.read_csv(csv_file)

# Iterate over each participant's data for the first 5 participants
for participant_id, participant_data in trial_info.groupby('Participant_id'):
    participant_id_str = f"s{participant_id:02d}"  # Convert participant_id to string format
    if participant_id_str not in participant_ids:
        continue

    # Get the raw EEG data for the current participant
    raw_data = eeg_data[participant_id_str]

    # Iterate over each trial for the current participant
    for index, trial in participant_data.iterrows():
        # Extract the start time of the trial (in seconds)
        start_time = trial['Start_time'] / 1e6  # Convert microseconds to seconds

        # Define the start and end time of the trial (assuming 1-minute duration)
        end_time = start_time + 60  # 1-minute duration

        # Extract the trial data based on the start and end time
        trial_data = raw_data.copy().crop(tmin=start_time, tmax=end_time)

        # Store the trial data in the list for the current participant
        participant_trials[participant_id_str].append(trial_data)

        # Print participant ID and trial information
        print(participant_id_str, trial)


s01 Participant_id          1.00
Trial                   1.00
Experiment_id           5.00
Start_time        1695918.00
Valence                 6.96
Arousal                 3.92
Dominance               7.19
Liking                  6.05
Familiarity             4.00
Name: 0, dtype: float64
s01 Participant_id          1.00
Trial                   2.00
Experiment_id          18.00
Start_time        2714905.00
Valence                 7.23
Arousal                 7.15
Dominance               6.94
Liking                  8.01
Familiarity             4.00
Name: 1, dtype: float64
s01 Participant_id          1.00
Trial                   3.00
Experiment_id           4.00
Start_time        3586768.00
Valence                 4.94
Arousal                 6.01
Dominance               6.12
Liking                  8.06
Familiarity             4.00
Name: 2, dtype: float64
s01 Participant_id          1.00
Trial                   4.00
Experiment_id          24.00
Start_time        4493800.00
Valence      

In [13]:
# Get the number of participants
num_participants = len(participant_trials)
print(f"Number of participants: {num_participants}")

# Get the number of trials for each participant
for participant_id, trials in participant_trials.items():
    num_trials = len(trials)
    print(f"Participant {participant_id}: Number of trials = {num_trials}")

Number of participants: 5
Participant s01: Number of trials = 40
Participant s02: Number of trials = 40
Participant s03: Number of trials = 40
Participant s04: Number of trials = 40
Participant s05: Number of trials = 40


In [ ]:
# Reorganize the trials based on Experiment_id from 1 to 40
sorted_trials = {}

# Iterate over the participant trials
for participant_id, trials in participant_trials.items():
    # Filter trial information DataFrame for the current participant
    participant_trial_info = trial_info[trial_info['Participant_id'] == participant_id]

    # Sort the trials based on the Experiment_id from the filtered DataFrame
    sorted_trials[participant_id] = sorted(
        trials,
        key=lambda x: int(participant_trial_info.loc[x.trial_number - 1, 'Experiment_id'])
    )
    print(f"Participant {participant_id}: Trials sorted successfully")

# Update the participant_trials dictionary with the sorted trials
# participant_trials = sorted_trials.copy()

# Update the participant_trials dictionary with the sorted trials
# participant_trials = sorted_trials.copy()

# Now participant_trials contains a list of trials for each participant, sorted by Experiment_id


In [14]:
# Accessing the trial information DataFrame for the first participant
first_participant_trial_info = trial_info[trial_info['Participant_id'] == 1]

# Print the information of each trial
for index, trial_data in first_participant_trial_info.iterrows():
    print(f"Trial {index + 1} info:")
    print(trial_data)


Trial 1 info:
Participant_id          1.00
Trial                   1.00
Experiment_id           5.00
Start_time        1695918.00
Valence                 6.96
Arousal                 3.92
Dominance               7.19
Liking                  6.05
Familiarity             4.00
Name: 0, dtype: float64
Trial 2 info:
Participant_id          1.00
Trial                   2.00
Experiment_id          18.00
Start_time        2714905.00
Valence                 7.23
Arousal                 7.15
Dominance               6.94
Liking                  8.01
Familiarity             4.00
Name: 1, dtype: float64
Trial 3 info:
Participant_id          1.00
Trial                   3.00
Experiment_id           4.00
Start_time        3586768.00
Valence                 4.94
Arousal                 6.01
Dominance               6.12
Liking                  8.06
Familiarity             4.00
Name: 2, dtype: float64
Trial 4 info:
Participant_id          1.00
Trial                   4.00
Experiment_id          24.00
St